# Resolviendo el sistema carbonato usando Python
****
Dr. Fernando J. Gomez (CICTERRA-CONICET-UNC)

## Introducción

Las ecuaciones que expresan el equilibrio del sistema carbonato, tal como se lo detalla a continuación, puede manipularse de forma de obtener un polinomio que, al resolverlo, nos permite calcular el pH y por consiguiente las concentraciones relativas de otros compuestos como ser $CO_2(aq)$, $HCO_3^-$ y $CO_3^{-2}$. Esto, junto a la concentración de $Ca^{+2}$ nos permite calcular el estado de saturación ($\Omega$) para diferentes minerales de carbonato de calcio como ser calcita ($\Omega_c$) y aragonita ($\Omega_a$).
En este ejemplo utilizaremos la función de **numpy** llamada *numpy.roots* (que en el código abreviamos como *np.roots*) para encontrar las raíces del polinomio de quinto orden que resulta de resolver el equilibrio del sistema carbonato en función del carbono inorngánico disuelto ($DIC$) y la alcalinidad ($ALK$). Crearemos una función que tiene como entrada o "input" la temperatura ($TK$) (en grados Kelvin), la salinidad ($S$), la profundidad ($z$), la alcalinidad ($ALK$) y el carbono inorgánico disuelto ($DIC$) y que nos permite calcular ("output") $fCO_{2}$, $pH$, $CO_2$, $HCO3^{-}$, $CO3^{-2}$, $H^{+}$, $OH^{-}$, $B(OH)_3$, $B(OH)_4^-$, y el *factor de Revelle* ($R$).

El ejemplo esta tomado de **Numerical Analysis with R: Solutions to ODEs and PDEs** de Graham W. Griffiths. En este libro el autor crea esta función en **R**, aqui simplemente trasladaremos dicha función (con ligeras modificaciones) en el lenguaje de **Python**.
La idea es mostrar como crear y utilizar funciones en **Python** y librerías de **numpy** para obtener las raíces (reales) del polinomio que nos permite obtener la concentración de $H^-$ y por consiguiente el $pH$ y los demás componentes del sistema carbonato ($CO_2$, $HCO3^{-}$, $CO3^{-2}$).

Las ecuaciones que dan bases a nuestro ejemplo son las siguientes (ver mas detalles en Griffiths, 2016):

Segun la Ley de Henry podemos calcular las concentraciones en equilibrio entre la presión parcial de $CO_2$ y la concentración de dióxido de carbono disuelto en el agua $[CO_2(aq)]$, donde $K_H$ es la constante de Henry (mol/kg/atm):

$$pCO_2=\frac{[CO_2(aq)]}{K_H}$$

Cuando el $CO_2(aq)$ se disuelve en el agua se generan las siguientes reacciones de equilibrio:

$$CO_2(aq) + H_2O \rightleftharpoons H_2CO_3$$

$$H_2CO_3 \rightleftharpoons HCO_3^- + H^+$$

$$HCO_3^- \rightleftharpoons CO_3^{-2} + H^+$$

En condiciones de equilibrio las concentraciones relativas estan controladas por las correspondientes constantes de equilibrio:

$$K_{CO_2(aq)}=\frac{[CO_2(aq)]}{H_2CO_3}$$

$$K_{H_2CO_3}=\frac{[H^+][HCO_3^-]}{H_2CO_3}$$

Usando $H_2CO_3$ para igualar las ecuaciones anteriores obtenemos la primera y segunda constante de disociación para el acido carbónico:

$$K_{a_1}=\frac{[H^+][HCO_3^-]}{CO_2(aq)}$$

$$K_{a_2}=\frac{[H^+][CO_3^{-2}]}{HCO_3^-}$$

Tambien es necesario considerar la disociación del agua y su consiguiente constante de disociación:

$$H^+ + OH^- \rightleftharpoons H_2O$$

$$K_W=[OH^-][H^+]$$


Otra variable necesaria es el carbono inorgánico disuelto ($DIC$):

$$DIC=[HCO_3^-]+[CO_3^{-2}]+[CO_2(aq)]$$

Al combinar las ecuaciones anteriores podemos obtener $[CO_2(aq)]$, $[HCO_3^-]$ y $[CO_3^{-2}]$:

$$[CO_2(aq)]= \frac{DIC}{1+\frac{K_{a1}}{[H^+]}+\frac{K_{a1}K_{a2}}{[H^+]^2}}$$


$$[HCO_3]= \frac{DIC}{\frac{[H^+]}{K_{a1}}+1+\frac{K_{a2}}{[H^+]}}$$

$$[CO_3^{-2}]= \frac{DIC}{1+\frac{[H^+]^2}{K_{a1}K_{a2}}+\frac{[H^+]}{K_{a2}}}$$

El ácido bórico tambien debe ser considerado, el mismo se disocia mediante la siguiente reacción:

$$B(OH)_3+H_2O \rightleftharpoons B(OH)_4^-+H^+$$

 y su equilibrio se encuentra controlado por la siguiente expresión donde K_B es la constante de equilibrio del ácido bórico:
 
 $$K_B=\frac{[B(OH)_4^-][H^+]}{[B(OH)_3]}$$
 
 La cantidad todal de boro $B_T$ es conservativa y depende de la salinidad:
 
 $$B_T=0.000416\frac{s}{35.0}$$
 
 Esto es igual a:
 
 $$B_T=[B(OH)_3]+[B(OH)_4^-]$$
 
 Ahora si estamos en condiciones de definir la alcalinidad total ($TA$) (esto despreciando otros compuestos menores que tambien contribuyen a la $TA$):
 
 $$TA=[HCO_3^-]+2[CO_3^{-2}]+[B(OH)_4^-]+[OH^-]-[H^+]$$


Ahora estamos en condiciones de obtener el polinomio de quinto orden (ver Bacastow y Keeling, 1973) que nos permite calcular la concentración de $H^+$ (y por consiguiente el $pH$):

$$0=\sum_{i=0}^5 a_i[H^+]^i$$

cuyos coeficientes son los siguientes:
Para $a_6$:

$$a_6=K_WK_{a1}K_{a2}K_{B}$$

Para $a_5$:

$$a_5=-2K_{a1} K_{a2} DIC  K_{B}-K_B B_T K_{a1} K_{a2} - K_W K_{a1} K_B+TA K_{a1} K_{a2}K_B-TAK_{a1}K_{a2}K_B-K_WK_{a1}K_{a2}$$

Para $a_4$:

$$a_4= -K_{a1}DICK_{B}-2K_{a1}K_{a2}DIC-K_BB_TK_{a1}-K_WK_{a1}+TAK_{a1}K_{a2}+K_{a1}K_{a2}K_{B}+TAK_{a1}K_{B}-K_WK_B$$

Para $a_3$:

$$a_3= -K_{a1}DICK_{B}-2K_{a1}K_{a2}DIC-K_BB_TK_{a1}-K_WK_{a1}+TAK_{a1}K_{a2}+K_{a1}K_{a2}K_{B}+TAK_{a1}K_{B}-K_WK_B$$

Para $a_2$:

$$a_2=K_{a1}+TA+K_B$$

Para $a_1$:

$$a_1=1$$


Para calcular el grado de saturación de la calcita o aragonita la reacción y la constante de equilibrio son (respectivamente):

$$CaCO_3 \rightleftharpoons Ca^{+2}+CO_3^{-2}$$

$$K'_{sp}=[Ca^{+2}][CO_3^{-2}]$$

El grado de saturacion esta representado por $\Omega$:

$$\Omega=\frac{[Ca^{+2}][CO_3^{-2}]}{K'_{sp}}$$

Si $\Omega$ es mayor a $1$ el sistema esta sobresaturado y si es menor a $1$ el sistema esta subsaturado en esos minerales. $K'_{sp}(cal)$ y $K'_{sp}(ar)$ son las constantes de equilibrio para la calcita y aragonita respectivamente.


## Implementando las ecuaciones con Python 
El primer paso sera crear la función, luego comprobar que nos da los resultados esperados y luego calcular los parámetros del sistema carbonato para un *set* de valores de $DIC$.

In [2]:
-

## Probando con un par de ejemplos

Una vez creada la función primero veremos si nos da valores
similares a los de Griffiths, para *input* carbEq ($TK$=293.15, $s$=35, $z$=0, $ALK$=2.300, $DIC$=1.970)
*Output* esperado: $fCO_2$ = 0.00028007, $pH$ = 8.1715, $CO_2$ = 9.0764e-006, $HCO_3^-$ = 0.0017, $CO_3^{-2}$ = 0.00020373

Pueden compararse los 5 primeros valores detallados a continuación con el output esperado.       

In [4]:
print(carbEq (293.15, 35, 0, 2.300, 1.970, 0.0103))

(0.00028007007970335009, 8.171545623563299, 9.0763555320580811e-06, 0.0017301985592125143, 0.00023072508525542773, 6.7368112011194785e-09, 5.6957740282050387e-06, 0.00031333776694037336, 0.00010266223305962662, 9.1849481830157149, 3.5154857664658365, 5.4381427417183188)


Luego la usaremos la función creada para valores de DIC entre 1000 y 2500 y generaremos una tabla con los valores. En este caso tambien incluiremos el calculo de saturacion de la calcita y aragonita expresado como $\Omega$:

In [81]:
%matplotlib inline
dic_values=[1.000, 1.100, 1.200, 1.300, 1.400, 1.500, 1.600,\
            1.700, 1.800, 1.900, 2.000, 2.100, 2.200, 2.300,\
            2.400, 2.500]

for dic in dic_values:
    fco2, pH, co2, hco3, co3, h, oh, BOH3, BOH4, R, omega_ar, omega_cal= carbEq(TK=293.15,\
                                        s=35, z=0, alk=2.300, dic=dic, Ca=0.0103)
    print ('|~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('|pH= %-3g CO2= %-3g HCO3-= %-3f CO3-2= %-3f' % (pH, co2, hco3, co3))
    print ('|omega_ar= %-3g omega_cal= %-3g' % (omega_ar, omega_cal))
    print ('|~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

|~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
|pH= 9.59079 CO2= 4.4382e-08 HCO3-= 0.000222 CO3-2= 0.000778
|omega_ar= 11.8513 omega_cal= 18.3329
|~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
|~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
|pH= 9.38775 CO2= 1.09814e-07 HCO3-= 0.000344 CO3-2= 0.000756
|omega_ar= 11.5114 omega_cal= 17.8072
|~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
|~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
|pH= 9.21142 CO2= 2.33237e-07 HCO3-= 0.000487 CO3-2= 0.000712
|omega_ar= 10.8547 omega_cal= 16.7912
|~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
|~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
|pH= 9.05831 CO2= 4.36406e-07 HCO3-= 0.000641 CO3-2= 0.000659
|omega_ar= 10.0346 omega_cal= 15.5227
|~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
|~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
|pH= 8.92103 CO2= 7.47205e-07 HCO3